# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 11:17:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.49       Driver Version: 457.49       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   47C    P8     1W /  N/A |    410MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2021년 SK Data**

In [5]:
lexicon_2021 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2021_sk.csv')

lexicon_2021.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,sk하이닉스,660,매일경제,2021010104,[반도체] 메모리값 강세 지속…파운드리 수요도 늘듯,http://news.mk.co.kr/newsRead.php?no=387&year=...,◆ 신년기획 2021 경제기상도 ◆\r\n\n\n이재용 삼성전자 부회장(왼쪽)이 지...,2021-01-04,4,124500,128000,120500,126000,7995016,0.063291,1,1,신년 기획 경제 기상도 이재용 삼성 전자 부회장 왼쪽 지난해 충남 온양 삼성 전자 ...


In [6]:
lexicon_2021[lexicon_2021['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2021.dropna(axis=0, inplace=True)
lexicon_2021 = lexicon_2021.reset_index(drop=True)
lexicon_2021.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
pos_lexicon_2021 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2021['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2021['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2021['Tokenization'][x].split())):
                    if lexicon_2021['Tokenization'][x].split()[w] == list(set(lexicon_2021['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2021['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2021['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2021 = pos_lexicon_2021.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2021['date'][x], positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2021['Tokenization'][x].split()))[z])))

***2021-01-04 Cosine Similarity between <가치 있는> & <언택트> : 0.5496575236320496
***2021-01-04 Cosine Similarity between <개선> & <향상> : 0.5878475308418274
***2021-01-04 Cosine Similarity between <도움이 되는> & <언택트> : 0.5378727316856384
***2021-01-04 Cosine Similarity between <우세> & <강세> : 0.5120405554771423
***2021-01-04 Cosine Similarity between <의심의 여지가 없는> & <언택트> : 0.529306173324585
***2021-01-04 Cosine Similarity between <중요한> & <가장> : 0.5371253490447998
***2021-01-04 Cosine Similarity between <증대> & <급증> : 0.5051040649414062
***2021-01-04 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2021-01-04 Cosine Similarity between <증대> & <확보> : 0.5161436796188354
***2021-01-04 Cosine Similarity between <증대> & <향상> : 0.6337747573852539
***2021-01-04 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2021-01-04 Cosine Similarity between <추진> & <사업> : 0.5213737487792969
***2021-01-04 Cosine Similarity between <개선> & <개선> : 0.9999999403953552
***2021-01-04 Cosine Similarity b

In [11]:
pos_lexicon_2021

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2021-01-04,1,가치 있는,언택트,0.549658,1
1,2021-01-04,1,개선,향상,0.587848,1
2,2021-01-04,1,도움이 되는,언택트,0.537873,1
3,2021-01-04,1,우세,강세,0.512041,1
4,2021-01-04,1,의심의 여지가 없는,언택트,0.529306,1
...,...,...,...,...,...,...
11458,2021-09-16,1935,추진,진행,0.535857,2
11459,2021-09-16,1935,추진,사업,0.521374,2
11460,2021-09-16,1935,해결,문제,0.618619,1
11461,2021-09-16,1935,해결,방안,0.531075,1


In [12]:
pos_lexicon_2021.to_csv('../../../../Code/Data/Test/Stock-Year/pos_sk_2021.csv', index=False)

In [13]:
a = list(set(list(pos_lexicon_2021['news_word'])))

a

['본격',
 '훈풍',
 '횡보할',
 '전개',
 '웃돈',
 '엠에스',
 '게임스탑',
 '밑돌',
 '김동천',
 '배팅했다',
 '최선',
 '급증',
 '코누코피아',
 '문제',
 '우세',
 '차익',
 '텔라세벡',
 '온택트란',
 '횡보세',
 '나정환',
 '빠듯하',
 '증가',
 '강창진',
 '진압되',
 '개인컴퓨터',
 '기록',
 '문제해결',
 '크래프칙',
 '상딩히',
 '로컬테인먼트',
 '추동훈',
 '강세',
 '검토',
 '단점',
 '고취',
 '펜데믹',
 '소재효',
 '간펑리튬',
 '들보',
 '강점',
 '세컨드폰',
 '이미드',
 '디지털리서치',
 '윤제용',
 '펀인터렉티브',
 '시너지',
 '가장',
 '닛케이지수',
 '해법',
 '기가톤',
 '박제완',
 '박유악',
 '리레이팅',
 '밸류맵',
 '급락',
 '중요',
 '아달리무맙',
 '박승영',
 '청주세무서',
 '퍼세컨드',
 '충족',
 '쇼와덴코',
 '폭락',
 '소형모듈원전',
 '상괄르',
 '우호',
 '한일병탄조약',
 '나성준',
 '최고급',
 '주요한',
 '머니무브',
 '류영재',
 '이유섭',
 '가팔랐으',
 '특성',
 '상쇄하',
 '빅테크주',
 '루시드모터스',
 '웃돈다',
 '리뉴텍',
 '적극',
 '증진',
 '밑돈',
 '커티스쿨릭',
 '발전',
 '활황',
 '송출되',
 '확보',
 '최준우',
 '수펙스',
 '저하',
 '팽팽하',
 '견조했',
 '박기효',
 '류기정',
 '신태수',
 '파이넌스',
 '샤오펑모터스',
 '크래프톤크래프톤',
 '센가쿠열',
 '곽병익씨',
 '기가와트',
 '맥쿼리인프라',
 '사하나',
 '불황',
 '박병창',
 '극복',
 '이점',
 '업황',
 '루브리컨츠',
 '빌드라잇양',
 '방법',
 '계획',
 '자이앤트',
 '확립',
 '실패',
 '감소',
 '빅테크',
 '특징',
 '장단점',
 '레그단비맙',
 '빅프라핏',


In [14]:
b = list(set(list(pos_lexicon_2021[pos_lexicon_2021['cosine_similarity']>=0.7]['news_word'])))

b

['밑돈', '불황', '경의', '충족', '추진', '실패', '단점', '약세', '최선', '장점', '개선']

In [15]:
c = list(set(list(pos_lexicon_2021[pos_lexicon_2021['cosine_similarity']>=0.65]['news_word'])))

c

['웃돌',
 '최선',
 '개선',
 '불황',
 '충족',
 '스타벤처스',
 '지성규',
 '장점',
 '증가',
 '경의',
 '실패',
 '단점',
 '약세',
 '빅테크주',
 '감소',
 '적극',
 '밑돈',
 '빅프라핏',
 '추진',
 '송출되']